In [1]:
from pprint import pprint
from datetime import datetime
from tqdm import tqdm
import matplotlib
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
from numpy.lib.stride_tricks import sliding_window_view
from utils import *

from IPython.display import display, Markdown, HTML 
import markdown2

#given a np array of shape (n,2) that contains intervals of time with an associated label (1-m) and a total duration of D seconds, how would I discretize this into an array with each element representing  elements using 0 to stand for values that were not in any interval, and the associated label to stand for any element that is 

In [2]:
# COMPUTE TRANSITION PROBABILITIES FOR EYES

def reduce_repeated_elements(arr):
    mask = np.concatenate(([True], np.diff(arr) != 0))
    reduced_arr = arr[mask]
    return reduced_arr

def compute_transition_probabilities(arr, normalized=True):
    num_integers = np.max(arr) + 1
    transition_matrix = np.zeros((num_integers, num_integers), dtype=[int,float][int(normalized)])
    # Create an array with shifted elements
    shifted_arr = np.roll(arr, -1)
    # Count occurrences of transitions
    unique_transitions, counts = np.unique(arr * num_integers + shifted_arr, return_counts=True)
    transition_matrix[unique_transitions // num_integers, unique_transitions % num_integers] = counts
    # Normalize transition matrix
    if normalized:
        transition_matrix /= np.sum(transition_matrix, axis=1, keepdims=True)
    return transition_matrix

def task_transition_matrix(eye_data, use_none=False, normalized=False):
    for i, (task, window_properties) in enumerate(sorted(ALL_WINDOW_PROPERTIES.items()), start=int(use_none)):
        intaskbox = in_box(eye_data['x'].to_numpy(), eye_data['y'].to_numpy(), window_properties['position'], window_properties['size'])
        eye_data.loc[intaskbox, 'task'] = i  
    task_data = reduce_repeated_elements(eye_data['task'].to_numpy())
    task_t_probs = compute_transition_probabilities(task_data, normalized=False)
    labels = (["none"] if use_none else []) + list(sorted(ALL_WINDOW_PROPERTIES.keys()))
    return pd.DataFrame(task_t_probs, index=labels, columns=labels)
    
data = load_nested_dict('data/Processed') 
for participant, _data1 in data.items(): #itertools.islice(data.items(), 1):
    for experiment, _data2 in _data1.items():
        start_time, finish_time =  _data2['start_time'], _data2['finish_time']
 
        eye_data = _data2['eye_tracking']  
        gaze = eye_data['gaze'].to_numpy().astype(bool)
        eye_data['task'] = np.zeros_like(eye_data['x'].to_numpy(), dtype=int)
        eye_data = eye_data.loc[gaze] # take only gaze data.                 
        t_probs = task_transition_matrix(eye_data, use_none=False, normalized=False)
        print(t_probs)
        #gaze = eye_data['gaze'].to_numpy()
    break

        

          fuel  system  tracking
fuel         0      24        38
system      20       0        29
tracking    42      25         0
          fuel  system  tracking
fuel         0      26        29
system      27       0        31
tracking    28      32         0
          fuel  system  tracking
fuel         0      25        42
system      27       0        27
tracking    40      29         1
          fuel  system  tracking
fuel         1      38        26
system      28       0        26
tracking    36      16         0


In [25]:

GAZE_ONLY = False 

def merge_intervals_nonoverlapping(intervals, labels=None):
    if labels is None:
        labels = np.arange(len(intervals))
    labels = np.concatenate([np.full((x.shape[0],1),i) for x, i in zip(intervals, labels)])
    intervals = np.concatenate(intervals, axis=0)
    indx = np.argsort(intervals[:,0]) # sort by start times
    intervals, labels = intervals[indx], labels[indx]
    
    return pd.DataFrame(dict(t1 = intervals[:,0], t2=intervals[:,1], task=labels[:,0]))
    
def in_intervals(intervals, points):
    return (points >= intervals[:, 0][:, np.newaxis]) & (points <= intervals[:, 1][:, np.newaxis])

def plot_intervals(intervals, ax, alpha=0.25, color='red'):
    for interval in intervals:
        ax.axvspan(*interval, alpha=alpha, color=color, linewidth=0)

def transition_pattern_count(data, n=2):
    tp = sliding_window_view(data, (n,)).squeeze()
    u, c = np.unique(tp, axis=0, return_counts=True)
    columns = [f"t{i}" for i in range(n)] + ["count"]
    return pd.DataFrame(data=np.concatenate([u,c[:,np.newaxis]], axis=1), columns=columns)


data = load_nested_dict('data/Processed') 
for participant, _data1 in data.items(): #itertools.islice(data.items(), 1):
    for experiment, _data2 in _data1.items():
        start_time, finish_time =  _data2['start_time'], _data2['finish_time']
        eye_data = _data2['eye_tracking']  
        gaze = eye_data['gaze'].to_numpy().astype(bool)
        #fig = plt.figure(figsize=(24,2))
        failure_intervals, look_intervals = [], []
        for task, _data3 in sorted(_data2['tasks'].items()):
            window_properties = ALL_WINDOW_PROPERTIES[task]
            # compute failure intervals
            failure_interval = merge_intervals([compute_time_intervals(x.failure, x.timestamp, start_time, finish_time).intervals for x in _data3.values()])
            failure_intervals.append(failure_interval)
            # compute gaze intervals
            intaskbox = in_box(eye_data['x'].to_numpy(), eye_data['y'].to_numpy(), window_properties['position'], window_properties['size'])
            if GAZE_ONLY: # USE THIS IF WE ONLY WANT GAZE
                intaskbox = np.logical_and(intaskbox, gaze) 
            look_interval = compute_time_intervals(intaskbox, eye_data['timestamp'].to_numpy(), start_time, finish_time, pad='next').intervals
            look_intervals.append(look_interval)
            #plot_intervals(look_interval, ax=plt.gca(), color=ALL_WINDOW_PROPERTIES[task]['color'])
        # t1, t2, task  
        look_intervals = merge_intervals_nonoverlapping(look_intervals)
        fdata = []
        # FAILED ALREADY | FAILED DURING | STILL FAILED
        for task, finterval in zip(sorted(_data2['tasks'].keys()), failure_intervals):
            # if the start of the gaze is within a failure interval, it was already failed when they looked.
            fstart = in_intervals(finterval, look_intervals['t1'].to_numpy()).any(axis=0)
            # if the start of the failure is within the gaze interval, it started during the gaze.
            fduring = in_intervals(look_intervals[['t1','t2']].to_numpy(), finterval[:,0]).any(axis=1) 
            # if the end of the gaze is in a failure interval, it continues after they look away.
            fend = in_intervals(finterval, look_intervals['t2'].to_numpy()).any(axis=0) 
            fdata.append(pd.DataFrame(dict(fstart=fstart, fduring=fduring, fend=fend)))
            

        # COMPUTE SOME STATS FOR THIS EXPERIMENT
        look_task = look_intervals['task'].to_numpy()
        # transition probabilities
        t_probs = compute_transition_probabilities(look_task, normalized=False)
        t_labels = list(sorted(ALL_WINDOW_PROPERTIES.keys()))
        print("--- Transition Table ---")
        print(pd.DataFrame(t_probs, index=t_labels, columns=t_labels))
        
        # compute most common patterns of length 2 and 3
        
        
        
        
            #size = data.max()
            #result = np.zeros(data.max(), data.max())
            #t_labels = list(sorted(ALL_WINDOW_PROPERTIES.keys()))
            #for _u, _c in zip(u,c):
            #    result[_u[0],_u[1]] = c
            #return datresult
        
        tr2 = transition_pattern_count(look_task, 2)
        tr3 = transition_pattern_count(look_task, 3)
        print(tr2)
        print(tr3)
        
        
        
        
        
        
        
        
        
         
            
        break
    break
    
            
            
          
           
        
            
           
    
        
        

--- Transition Table ---
          fuel  system  tracking
fuel         9      12        28
system      12       5        22
tracking    28      22         1
   t0  t1  count
0   0   0      9
1   0   1     12
2   0   2     28
3   1   0     12
4   1   1      5
5   1   2     21
6   2   0     28
7   2   1     22
8   2   2      1
    t0  t1  t2  count
0    0   0   0      3
1    0   0   1      1
2    0   0   2      5
3    0   1   0      4
4    0   1   1      3
5    0   1   2      5
6    0   2   0     10
7    0   2   1     18
8    1   0   0      1
9    1   0   1      4
10   1   0   2      7
11   1   1   2      5
12   1   2   0     16
13   1   2   1      4
14   1   2   2      1
15   2   0   0      5
16   2   0   1      7
17   2   0   2     16
18   2   1   0      8
19   2   1   1      2
20   2   1   2     11
21   2   2   0      1
    t0  t1  t2  t3  count
0    0   0   0   0      2
1    0   0   0   2      1
2    0   0   1   2      1
3    0   0   2   0      2
4    0   0   2   1      3
5    0   1 